In [1]:
from scnn.bipolar_functions.operations import *
from scnn.convertors.sng import *
import numpy as np
import matplotlib.pyplot as plt

In [3]:
f2s = F2S(seq_len=1024)

In [20]:
sample = f2s(torch.rand(128)*2-1)

In [5]:
sample.shape

torch.Size([128, 1024])

In [7]:
class ap_counter():
    def __init__(self, in_features=16, num_au_layers=1):
        self.num_au_layers = num_au_layers
        self.in_features = in_features

    def approx_unit(self, x: torch.tensor) -> torch.tensor:
        seq_len = x.shape[-1]
        half = int(seq_len/2)
        # assert x.shape[-2] <= self.in_features
        # if x.shape[-2] < self.in_features:
        #     x = torch.cat([x, torch.zeros(x.shape[:-2]+(self.in_features-x.shape[-2],seq_len,), dtype=torch.bool)],dim=-2)

        idx1, idx2, idx3, idx4 = [torch.arange(i, x.shape[-2], 4) for i in range(4)]
        grp1, grp2, grp3, grp4 = x[...,idx1,:], x[...,idx2,:], x[...,idx3,:], x[...,idx4,:]

        
        out1 = torch.concat([torch.logical_or(grp1[...,:half],grp2[...,:half]), torch.logical_and(grp3[...,:half],grp4[...,:half])], dim=-2)
        out2 = torch.concat([torch.logical_and(grp1[...,half:],grp2[...,half:]), torch.logical_or(grp3[...,half:],grp4[...,half:])], dim=-2)
        
        
        return torch.concat([out1,out2],dim=-1)
    
    def counter(self, x):
        return torch.sum(x, dim=-2, dtype=torch.int16)
    
    def __call__(self, x):
        count = 0
        for i in range(self.num_au_layers):
            num_units = int(x.shape[-2]/4)
            x, res = x[...,0:4*num_units,:], x[...,4*num_units:,:] 
            x = self.approx_unit(x)
            count += torch.tensor(2**i, dtype=torch.int16)*self.counter(res)
        return torch.tensor(2**self.num_au_layers, dtype=torch.int16)*self.counter(x)+count

In [14]:
apc_1=ap_counter(in_features=128, num_au_layers=1)
apc_0=ap_counter(in_features=128, num_au_layers=0)

In [31]:
sample = f2s(torch.rand(128)*2-1)
(apc_1(sample).sum()-apc_0(sample).sum())/apc_0(sample).sum()

tensor(0.0025)

tensor(66536)

tensor(-0.0197)